In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# 📌 Load Data with Fixes
def load_data(file_path):
    df = pd.read_csv(file_path)

    # 🔹 Clean column names (remove spaces, lowercase everything)
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    print("📌 Available Columns in Data:", df.columns.tolist())  # Debugging: Show correct column names
    return df

# 🔍 **Improved Function to Find Closest Column Name**
def get_closest_column(df, target_column):
    column_map = {
        "foreign_exchange_reserves": "foreign_exchange_reserves_(billion_usd)",
        "exports": "exports_(billion_usd)",
        "imports": "imports_(billion_usd)",
        "trade_balance": "trade_balance_(exports_-_imports)_(billion_usd)"
    }

    if target_column in column_map:
        correct_col_name = column_map[target_column]
        if correct_col_name in df.columns:
            return correct_col_name
    print(f"⚠ Warning: '{target_column}' not found! Available columns: {df.columns.tolist()}")
    return None

# 📊 **Trend Analysis Plot**
def plot_trend_analysis(df, save_path="trend_analysis.png"):
    forex_col = get_closest_column(df, "foreign_exchange_reserves")
    exports_col = get_closest_column(df, "exports")
    imports_col = get_closest_column(df, "imports")
    trade_balance_col = get_closest_column(df, "trade_balance")

    if not all([forex_col, exports_col, imports_col, trade_balance_col]):
        print("⚠ Missing required columns for trend analysis. Skipping plot.")
        return None

    plt.figure(figsize=(12, 6))
    plt.plot(df['year'], df[forex_col], label='Forex Reserves', marker='o')
    plt.plot(df['year'], df[exports_col], label='Exports', marker='s')
    plt.plot(df['year'], df[imports_col], label='Imports', marker='^')
    plt.plot(df['year'], df[trade_balance_col], label='Trade Balance', marker='d')

    plt.xlabel("Year")
    plt.ylabel("Billion USD")
    plt.title("📈 Foreign Exchange & Trade Trends (1980-2024)", fontsize=14, fontweight="bold", color="darkblue")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.savefig(save_path)
    plt.close()
    return save_path  # Return saved image path

# 📊 **Trade Balance Bar Chart**
def plot_trade_balance_bar(df, save_path="trade_balance_bar.png"):
    trade_balance_col = get_closest_column(df, "trade_balance")

    if not trade_balance_col:
        print("⚠ 'Trade Balance' column is missing. Skipping bar chart.")
        return None

    plt.figure(figsize=(10, 5))
    plt.bar(df['year'], df[trade_balance_col], color='skyblue')
    plt.xlabel("Year")
    plt.ylabel("Trade Balance (Billion USD)")
    plt.title("📊 Trade Balance Over Years (Bar Chart)")
    plt.grid(axis='y', linestyle="--", alpha=0.5)
    plt.savefig(save_path)
    plt.close()
    return save_path

# 🔥 **Trade Balance Correlation Heatmap**
def plot_trade_balance_heatmap(df, save_path="trade_balance_heatmap.png"):
    if 'year' not in df.columns:
        print("⚠ 'Year' column is missing. Skipping heatmap.")
        return None

    plt.figure(figsize=(10, 5))
    sns.heatmap(df.set_index('year').corr(), annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title("🔥 Trade Balance Correlation Heatmap", fontsize=14, fontweight="bold", color="darkred")
    plt.savefig(save_path)
    plt.close()
    return save_path

# 📌 **Generate Stylish PDF Report**
def generate_pdf_report(df, trend_path, bar_path, heatmap_path, start_year, end_year):
    report_path = f"Trade_Balance_Report_{start_year}_to_{end_year}.pdf"
    doc = SimpleDocTemplate(report_path, pagesize=letter)
    elements = []

    # **Title Styling**
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        "TitleStyle",
        parent=styles["Title"],
        fontSize=18,
        textColor=colors.darkblue,
        spaceAfter=10,
        alignment=1  # Center Align
    )

    # **Add Title**
    elements.append(Paragraph(f"📈 Trade Balance Report ({start_year}-{end_year})", title_style))
    elements.append(Spacer(1, 12))

    # **Create Table**
    forex_col = get_closest_column(df, "foreign_exchange_reserves")
    exports_col = get_closest_column(df, "exports")
    imports_col = get_closest_column(df, "imports")
    trade_balance_col = get_closest_column(df, "trade_balance")

    col_headers = ["Year", "Forex Reserves (Billion USD)", "Exports", "Imports", "Trade Balance"]
    data = [col_headers]

    df_filtered = df[(df["year"] >= start_year) & (df["year"] <= end_year)]
    for _, row in df_filtered.iterrows():
        data.append([row["year"], row[forex_col], row[exports_col], row[imports_col], row[trade_balance_col]])

    table = Table(data)
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.darkblue),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 1), (-1, -1), 9),
        ("BACKGROUND", (0, 1), (-1, -1), colors.lightgrey),
        ("GRID", (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    elements.append(Spacer(1, 20))

    # **Insert Visuals**
    if trend_path:
        elements.append(Image(trend_path, width=400, height=250))
    if bar_path:
        elements.append(Image(bar_path, width=400, height=250))
    if heatmap_path:
        elements.append(Image(heatmap_path, width=400, height=250))

    # **Build PDF**
    doc.build(elements)
    print(f"✅ PDF report saved: {report_path}")
    return report_path

# **Run the Program**
if __name__ == "__main__":
    file_path = "Foreign_Exchange_Trade_Balance_India_1980_2024.csv"
    df = load_data(file_path)  # Load Data with Fixed Column Names

    # **User Inputs**
    start_year = int(input("📅 Enter start year for report: "))
    end_year = int(input("📅 Enter end year for report: "))

    # **Generate Visuals**
    trend_path = plot_trend_analysis(df)
    bar_path = plot_trade_balance_bar(df)
    heatmap_path = plot_trade_balance_heatmap(df)

    # **Generate PDF Report**
    pdf_report = generate_pdf_report(df, trend_path, bar_path, heatmap_path, start_year, end_year)

    print("📊 Download report:", pdf_report)


📌 Available Columns in Data: ['year', 'foreign_exchange_reserves_(billion_usd)', 'exports_(billion_usd)', 'imports_(billion_usd)', 'trade_balance_(exports_-_imports)_(billion_usd)']


C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_14100\1336844264.py:56: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(save_path)
C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_14100\1336844264.py:74: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(save_path)
C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_14100\1336844264.py:87: UserWarning: Glyph 128293 (\N{FIRE}) missing from font(s) DejaVu Sans.
  plt.savefig(save_path)


✅ PDF report saved: Trade_Balance_Report_2000_to_2023.pdf
📊 Download report: Trade_Balance_Report_2000_to_2023.pdf
